In [2]:
from collections import Counter
import os
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
    print(len(emails))
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:
                    words = line.split()
                    all_words += words
    
    dictionary = Counter(all_words)
    # Paste code for non-word removal here

    list_to_remove = dictionary.keys()
    for item in  list(list_to_remove):
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]

    # get the most common word
    dictionary = dictionary.most_common(3000)

    return dictionary

In [3]:
dictionary = make_Dictionary('./ling-spam/train-mails')

702


In [4]:
import numpy as np
def extract_features(mail_dir):
    files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 3000))
    features_y =[]
    docID = 0
    for fil in files:
        if 'spmsg' in fil:
            features_y.append(1)
        else:
            features_y.append(0)

        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i, d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID, wordID] = 1
            docID += 1

    return features_matrix, np.array(features_y)

In [4]:
# get train features and class
x_train, y_train = extract_features('./ling-spam/train-mails')

In [5]:
train_num = len(y_train)
features_num = len(x_train[0])
p_spmsg = sum(y_train)/len(y_train)
p_msg = 1 - p_spmsg

p_w_spmsg = np.ones(features_num)
p_w_msg = np.ones(features_num)

p_w_spmsg_sum = 0
p_w_msg_sum = 0

for i in range(train_num):
    if(y_train[i] == 1):
        p_w_spmsg += x_train[i]
        p_w_spmsg_sum += sum(x_train[i])
    else:
        p_w_msg += x_train[i]
        p_w_msg_sum += sum(x_train[i])

# calculate p(w|c)
# zero conditional probability
# use log change add to multiply
# p_w_spmsg = np.log(p_w_spmsg+1*1/2)/(p_w_spmsg_sum+1)
# p_w_msg = np.log(p_w_msg+1*1/2)/(p_w_msg_sum+1)
p_w_spmsg = np.log(p_w_spmsg/p_w_spmsg_sum)
p_w_msg = np.log(p_w_msg/p_w_msg_sum)

In [6]:
# get test features and class
x_test, y_test = extract_features('./ling-spam/test-mails')

In [1]:
# calculate accuracy, precision, recall, fscore
p_spmsg_w = x_test @ p_w_spmsg + np.log(p_spmsg)
p_msg_w = x_test @ p_w_msg + np.log(p_msg)
y_predict = (p_spmsg_w > p_msg_w)+0

accuracy = sum(y_predict == y_test)/len(y_test)

TP = sum(y_predict & y_test)
FN = sum(y_predict ^ np.ones(len(y_predict)))
precision = TP/sum(y_predict)
recall = TP/(TP+FN)
fscore = (1 + 0.5**2) * (precision * recall) / ((0.5**2 * precision) + recall)
print('accuracy:{:.4f} precision:{:.4f} recall:{:.4f} fscore:{:.4f}'.format(accuracy, precision, recall, fscore))

NameError: name 'x_test' is not defined